In [1]:
import numpy as np
import re
import string
import time

In [2]:
class Simplex:
    def __init__(self,tipo='feasibility'):
        # VARIÁVEIS INICIAIS: Tipo de otimização
        if tipo not in ['feasibility','max','min']:
            raise RuntimeError('Wrong type of objective (must be feasibility, max, min)')
        # inicializar as variáveis internas à classe
        self.tipo = tipo
        self.c = np.array([])
        self.A = np.array([])
        self.b = np.array([]).reshape(-1,1)
        self.var_names = {}
        self.var_types = {}
        self.res_type = []
            
    def add_vars(self,variables):
        # ENTRADA: lista de strings da forma 'nomeVariavel tipoVariavel' onde 
        # tipo pertence a ['positive','negative','real']
        
        # SAÍDA: _
        
        # DESCRIÇÃO:
        # guardar internamente na variável var_names e var_types os nomes e tipos de cada variável
        
        for var in variables:
            if (' ' not in var) or (var not in self.var_names.keys()):
                var = var.split()
                self.var_names[var[0]] = len(self.var_names.keys())
                self.var_types[var[0]] = var[1]
            else:
                raise RuntimeError
        self.A = self.A.reshape(-1,len(self.var_names))
            
    def add_rest(self,restrictions):
        # ENTRADA: lista de strings da forma de igualdades/desigualdades lineares 
        
        # SAÍDA: _
        
        # DESCRIÇÃO:
        # processar individualmente cada restrição e string de modo a gerar um array de coeficientes
        # com os valores corretos para cada variável
        
        for ind, rest in enumerate(restrictions):
            print('Adding restriction '+rest+' ...')
            symbols = ['<=','>=','=']                                        # tipos de rest
            var_names = self.var_names   
            symbol = [symbol for symbol in symbols if symbol in rest][0]     # identificar o tipo de rest
            rest = re.sub(r'\s+', '', rest)                                  # sumir com espaços
            rest = '+-'.join(rest.split(sep = '-'))                          # adicionar + antes de -
            left, right = rest.split(sep = symbol)                           # dividir a equação em dois lados
            left = [word.strip() for word in left.split(sep = '+')]          # particionar cada lado
            right = [word.strip() for word in right.split(sep = '+')]
            all = left+right
            var_coef = np.zeros(shape=[1,len(var_names)])                    # declarar o array de coef
            b = np.zeros(shape=(1,1))                                        # declarar o valor de b
            for cell in all:         # para cada célula identificar a variável e o valor acompanhado
                isRight = cell in right
                if cell == '': continue
                has_var = [True for var_name in var_names.keys() if var_name in '-'.join(cell.split('*')).split('-')]
                if len(has_var)==1:
                    for var_name in var_names.keys():
                        if var_name in cell:
                            var_ind = var_names[var_name]
                            if '*' in cell:
                                var_coef[0,var_ind] = var_coef[0,var_ind] + float(cell.split(sep='*')[0]) * (1 - 2 * isRight)
                            elif '-' in cell:
                                var_coef[0,var_ind] = var_coef[0,var_ind] - 1 + (2 * isRight)
                            else:
                                var_coef[0,var_ind] = var_coef[0,var_ind] + 1 - (2 * isRight)
                            continue
                else:
                    b = b + float(cell) * (-1 + 2 * isRight)

            self.res_type.append(symbol)                         # adicionar os dados aos valores internos da classe

            self.A = np.concatenate([self.A, var_coef], axis=0)
            self.b = np.concatenate([self.b, b], axis=0)
            print('Done')      
  
    def add_obj(self,obj):
        # ENTRADA: string da forma de uma função linear
        
        # SAÍDA: _
        
        # DESCRIÇÃO:
        # semelhante a função anterior, processamos a string do objetivo de modo a parsear os coeficientes
        # e adicionar as variáveis internas da classe
        
        print('Adding objective '+obj+' ...')
        var_names = self.var_names
        obj = re.sub(r'\s+', '', obj)
        obj = '+-'.join(obj.split(sep = '-'))
        obj = obj.split(sep = '+')
        var_coef = np.zeros(shape=[1,len(var_names)])
        for cell in obj:
            if cell == '': continue
            has_var = [True for var_name in var_names.keys() if var_name in cell]
            if len(has_var)==1:
                for var_name in var_names.keys():
                    if var_name in cell:
                        var_ind = var_names[var_name]
                        if '*' in cell:
                            var_coef[0,var_ind] = var_coef[0,var_ind] + float(cell.split(sep='*')[0])
                        elif '-' in cell:
                            var_coef[0,var_ind] = var_coef[0,var_ind] - 1
                        else:
                            var_coef[0,var_ind] = var_coef[0,var_ind] + 1
        if self.tipo == 'min':
            var_coef = -1*var_coef
        self.c = var_coef
        print('Done')

    def solve_simplex(self,A,b,c,slack,artificial,basis,phase1=False,verbose=False):
        # ENTRADA:
        # A: matriz m x n correspondente aos coeficientes das m restrições e n variáveis
        # b: vetor coluna m x 1 correspondente aos coeficientes do valor de cada igualdade
        # c: vetor linha 1 x n correspondente aos coeficientes da função objetivo para cada variável 
        # slack: lista de indíces das variáveis slack
        # artificial: lista de indíces das variáveis artificiais
        # basis: lista de indíces das variáveis pertencentes à base
        # phase1: booleano indicando se há necessidade da fase 1
        # verbose: booleano indicando se desejamos ver os valores por iteração
        
        # SAÍDA: 
        # A: modificada pelo tableau, b modificado pelo tableau, c modificado pelo tableau
        # basis: correspondente aos indices das variáveis da basis do simplex
        # status: string declarando a situação do simplex
        # x: valor de x na última iteração do simplex
        # obj: valor do(s) objetivo(s) na última iteração do simplex
        
        # DESCRIÇÃO:
        # Como descrito no AMP capítulo 2, segue a implementação do algoritmo simplex por meio de duas fases
        # usando a estrutura tableau
        
        A, b, c, slack, artificial, basis = A.copy(), b.copy(), c.copy(), slack.copy(), artificial.copy(), basis.copy()
        status = 'running'
        
        # montar o x inicial para calcular o objetivo correspondente
        x = np.zeros(shape=(A.shape[1],1))
        for idx, var in enumerate(basis):
            x[var,0] = b[idx,0]
        
        # monta o objetivo (possui tamanho 2 caso haja duas fases)
        obj = [-1*np.dot(c_i,x) for c_i in c]

        # consertar c de modo a ter valor zero nas variáveis da base
        for var in basis:
            if c[0][0,var] != 0:
                row = np.argmax(A[:,var])
                c[0] = c[0] - c[0][0,var] * A[row,:]
        
        # variáveis úteis na inicialização do algoritmo
        cur_c = 0     # objetivo atual a minimizar
        n_ite = 0     # numero de iterações
        pivot_change = []   # troca de pivos (usado na remoção das artificiais pós fase 1)
        drop_art = False    # booleana indicadora de se deve remover as variáveis artificiais
        while status == 'running':
            n_ite += 1

            # printar variáveis caso verbose
            var_dic = {'Iteration: ':n_ite, 'A: ':A, 'b: ':b, 'c: ':c, 'slack: ':slack, 'A: ':artificial, 'basis: ':basis,
                     'objective: ':obj, 'phase1: ':2-phase1, 'pivot_change: ':pivot_change}
            if verbose:
                for name, var in var_dic.items():
                    print(name)
                    print(var)
                print('--------------------')
                
            # variáveis artificiais na base
            art_in_basis = [var for var in artificial if var in basis]
                
            # verifica se há alguma variável com coeficiente positivo em c
            any_pos_c = bool(sum(c[cur_c].ravel()>0)>0)
            if not any_pos_c:
                if not phase1:
                    # optimal    
                    status = 'success'
                else:
                    if obj[cur_c] != 0:
                        # unfeasible
                        status = 'unfeasible'
                    else:
                        # trocar de fase 1 para fase 2
                        # checar se há artificais na base e guardar as mudanças necessárias
                        drop_art = True
                        cur_c = 1
                        if len(art_in_basis)>0:
                            for var in art_in_basis:
                                row = int(np.argmax(A[:,var]))
                                row_idxs = np.arange(A.shape[1])[A[row,:]!=0]
                                if len(row_idxs) == 0:
                                    raise NameError('Ainda a implementar artificiais na base sem outros positivos')
                                new_pivot = [idx for idx in row_idxs if idx not in basis]
                                new_pivot = new_pivot[0]
                                pivot_change.append([var,new_pivot])

            else:
                # caso o problema seja de feasibility e já encontramos uma solução feasible
                if len(art_in_basis) == 0 and self.tipo == 'feasibility':
                    status = 'feasible'
                    break
                # iterar
                new_pivot = int(np.argmax(c[cur_c],axis=1))    # novo pivô
                # a_pos são os indices dos coeficientes maiores que 0 da coluna do pivô
                a_pos = np.arange(A.shape[0])[(A[:,new_pivot] > 0).ravel()]   
                if len(a_pos) == 0 and len(pivot_change) == 0:
                    status = 'unbounded'
                else:
                    if len(pivot_change) == 0 and drop_art == False:
                        # teste de ratio
                        ratios = [b[a,0]/A[a,new_pivot] for a in a_pos]
                        old_pivot = a_pos[int(np.argmin(ratios))]
                        pivot_change.append([basis[old_pivot],new_pivot])
                    # troca de pivos
                    for old_pivot, new_pivot in pivot_change:
                        # troca os indices na base
                        if old_pivot not in basis:
                            continue
                        old_pivot = basis.index(old_pivot)
                        basis[old_pivot] = new_pivot
                        # update A e B para ter valor 1 nesse coeficiente
                        k = 1/A[old_pivot,new_pivot]
                        A[old_pivot,:] = k*A[old_pivot,:]
                        b[old_pivot,0] = k*b[old_pivot,0]
                        for row in range(0,A.shape[0]):
                            if row != old_pivot:
                                k = A[row,new_pivot]
                                A[row,:] = A[row,:] - k * A[old_pivot,:]
                                b[row,0] = b[row,0] - k * b[old_pivot,0]
                        # update c e objetivo com essa nova troca de pivos
                        for i in range(0+cur_c,len(c)):
                            k = c[i][0,new_pivot]
                            c[i][0,:] = c[i][0,:] - k * A[old_pivot,:]
                            obj[i] = obj[i] - k * b[old_pivot,0]
                
                # caso se tenha trocado de fase, remover as artificiais
                pivot_change = []
                if drop_art == True:
                    if self.tipo == 'feasibility':
                        status = 'feasible'
                    A = np.delete(A, np.s_[artificial], axis=1)
                    c[1] = np.delete(c[1], np.s_[artificial], axis=1)
                    # corrigir index das outras variaveis
                    for i in range(len(artificial)-1,-1,-1):
                        cur = artificial[i]
                        basis = [var-1 if var>cur else var for var in basis]
                        slack = [var-1 if var>cur else var for var in slack]
                    artificial = []  
                    # corrigir c para não ter ninguém da base
                    for var in basis:
                        if c[1][0,var] != 0:
                            row = np.argmax(A[:,var])
                            c[1] = c[1] - c[1][0,var] * A[row,:]     
                    phase1 = False
                    drop_art = False
    
        self.n_ite = n_ite
        
        # montar o x final a partir do resultado do simplex
        x = np.zeros(shape=(A.shape[1],1))
        for idx, var in enumerate(basis):
            x[var,0] = b[idx,0]

        # corrigir o objetivo
        obj = [-1*obj_i for obj_i in obj]
        
        # print final caso verboso
        var_dic = {'Iteration: ':'FINAL', 'A: ':A, 'b: ':b, 'c: ':c, 'slack: ':slack, 'A: ':artificial, 'basis: ':basis,
                   'objective: ':obj, 'phase1: ':2-phase1, 'pivot_change: ':pivot_change}
        if verbose:
            for name, var in var_dic.items():
                print(name)
                print(var)
            print('--------------------')

        return A, b, c, basis, status, x, obj
      
    def solve(self,verbose):
        # ENTRADA: verbose: variável booleana, caso true o algoritmo mostra os valores entre iterações
        
        # SAÍDA: 
        # x_final: valor das variáveis obtido pela otimização via simplex
        # obj: valor do objetivo obtido pela otimização via simplex 
        
        # DESCRIÇÃO:
        # Essa função tem como objetivo montar o problema em forma canônica e controlar o fluxo de
        # sua execução
        
        tipo, c, A, b, var_names, var_types = self.tipo, self.c, self.A, self.b, self.var_names, self.var_types
        var_idx = {}

        print('Splitting into positive and negative...')
        # dividir em positivos e negativos, guardando os indices para futuro retorno
        n_total = len(var_names)
        for idx, var in enumerate(var_types.keys()):
            var_type = var_types[var]
            if var_type == 'negative':
                A[:,idx] = -1*A[:,idx]
                c[:,idx] = -1*c[:,idx]
            elif var_type == 'real':
                var_idx[var] = n_total
                n_total += 1
                A = np.concatenate([A, -1*A[:,idx].reshape(-1,1)],axis=1)
                c = np.concatenate([c, -1*c[:,idx].reshape(-1,1)],axis=1)
        print('Done')

        # adicionado slacks e artificiais conforme precise 
        print('Adding slack/artificial variables...')
        slack = []        # lista dos índices das variáveis slacks
        artificial = []   # lista dos índices das variáveis artificiais
        basis = []        # lista dos índices das variáveis pertencentes a base
        for idx, res_sym in enumerate(self.res_type):
            sym_change = {'=':'=', '>=':'<=', '<=':'>='}
            if b[idx,0] < 0:
                b[idx,0] = -1 * b[idx,0]
                A[idx,:] = -1 * A[idx,:]
                res_sym = sym_change[res_sym]

            if res_sym == '=':
                new_col = np.zeros(shape=(A.shape[0],1))
                new_col[idx,0] = 1
                artificial.append(A.shape[1])
                basis.append(A.shape[1])
                c = np.concatenate([c, np.zeros(shape=(1,new_col.shape[1]))],axis=1)
                A = np.concatenate([A, new_col],axis=1)
            elif res_sym == '<=':
                new_col = np.zeros(shape=(A.shape[0],1))
                new_col[idx,0] = 1
                slack.append(A.shape[1])
                basis.append(A.shape[1])
                c = np.concatenate([c, np.zeros(shape=(1,new_col.shape[1]))],axis=1)
                A = np.concatenate([A, new_col],axis=1)
            else:
                new_cols = np.zeros(shape=(A.shape[0],2))
                new_cols[idx,0] = -1
                new_cols[idx,1] = 1
                slack.append(A.shape[1])
                artificial.append(A.shape[1]+1)
                basis.append(A.shape[1]+1)
                c = np.concatenate([c, np.zeros(shape=(1,new_cols.shape[1]))],axis=1)
                A = np.concatenate([A, new_cols],axis=1)
        print('Done')

        # executar o simplex de acordo com a necessidade de fase 1
        phase1 = len(artificial) > 0
        # objetivo w ou z
        if phase1:
            print('Setting Phase 1 basis and Phase 2...')
            c_p1 = np.zeros(shape = (1,A.shape[1]))
            for art in artificial:
                c_p1[0,art] = -1
            c = [c_p1,c]
        else:
            print('Setting Phase 2 basis...')
            c = [c]
        print('Done')
        
        # executar simplex
        print('Starting simplex...')
        cur_time = time.time()
        A, b, c, basis, status, x, obj = self.solve_simplex(A,b,c,slack,artificial,basis,phase1,verbose=verbose)
        self.running_time = time.time() - cur_time
        print('Simplex finished! ' + str(self.n_ite) + ' iterations with ' + str(round(self.running_time,3)) + 's to run.')
        print('Simplex Status:',status)
        if status != 'success' and status != 'feasible':
            print('ERROR: '+status)
            return 0, 0

        print('Switching back to original x values...')
        # retornar aos valores originais das variáveis
        x = x[0:n_total,0].reshape(-1,1)
        if tipo == 'min':
            obj = -1*obj
        x_final = np.zeros(shape=(len(var_names),1))
        for idx, var in enumerate(var_names.keys()):
            var_type = var_types[var]
            if var_type == 'negative':
                x_final[idx, 0] = - x[idx, 0]
            elif var_type == 'real':
                x_final[idx, 0] = x[idx, 0] - x[var_idx[var], 0]
            else:
                x_final[idx, 0] = x[idx, 0]
                
        # resultado final
        print('Done')
        print('Final result:')
        print('X:')
        print(x_final)
        print('Objective Value:')
        print(obj)
        return x_final, obj

In [3]:
# testando o simplex

sim = Simplex('min')                                                      
sim.add_vars(['x1 positive','x2 positive','x3 positive'])   
sim.add_rest(['x1 - 2*x2 + x3 = 2'])                                    
sim.add_rest(['- x1 + 3*x2 - 2*x3 = -3'])
sim.add_obj('x1 + 3*x2 + 2*x3')
x_final, obj = sim.solve(verbose = False)

Adding restriction x1 - 2*x2 + x3 = 2 ...
Done
Adding restriction - x1 + 3*x2 - 2*x3 = -3 ...
Done
Adding objective x1 + 3*x2 + 2*x3 ...
Done
Splitting into positive and negative...
Done
Adding slack/artificial variables...
Done
Setting Phase 1 basis and Phase 2...
Done
Starting simplex...
Simplex finished! 5 iterations with 0.0s to run.
Simplex Status: success
Switching back to original x values...
Done
Final result:
X:
[[1.]
 [0.]
 [1.]]
Objective Value:
[]


In [4]:
# testando o simplex

sim = Simplex('min')                                                      
sim.add_vars(['x1 positive','x2 positive','x3 positive'])   
sim.add_rest(['2*x1 + 4*x2 + 2*x3 = 2'])                                    
sim.add_rest(['- x1 - 3*x2 - 2*x3 = 0'])
sim.add_obj('27*x1 + 2*x2 - 6*x3')
x_final, obj = sim.solve(verbose = False)

Adding restriction 2*x1 + 4*x2 + 2*x3 = 2 ...
Done
Adding restriction - x1 - 3*x2 - 2*x3 = 0 ...
Done
Adding objective 27*x1 + 2*x2 - 6*x3 ...
Done
Splitting into positive and negative...
Done
Adding slack/artificial variables...
Done
Setting Phase 1 basis and Phase 2...
Done
Starting simplex...
Simplex finished! 2 iterations with 0.001s to run.
Simplex Status: unfeasible
ERROR: unfeasible


In [5]:
# testando o simplex

sim = Simplex('min')                                                      
sim.add_vars(['x1 positive','x2 positive','x3 positive','x4 positive','x5 positive'])   
sim.add_rest(['x1 + x2 + x3 - x4 = 1'])                                    
sim.add_rest(['x2 + 2*x3 + x5 = 2'])
sim.add_obj('x2 - x4')
x_final, obj = sim.solve(verbose = False)

Adding restriction x1 + x2 + x3 - x4 = 1 ...
Done
Adding restriction x2 + 2*x3 + x5 = 2 ...
Done
Adding objective x2 - x4 ...
Done
Splitting into positive and negative...
Done
Adding slack/artificial variables...
Done
Setting Phase 1 basis and Phase 2...
Done
Starting simplex...
Simplex finished! 4 iterations with 0.0s to run.
Simplex Status: unbounded
ERROR: unbounded


In [6]:
# testando o simplex

sim = Simplex('min')                                                      
sim.add_vars(['x1 positive','x2 positive','x3 positive','x4 positive','x5 positive'])   
sim.add_rest(['-x1 + x2 - x5 = 1'])                                    
sim.add_rest(['x1 - x3 + x4 + 2*x5 = 1'])
sim.add_obj('2*x1 + 2*x3 + 3*x5')
x_final, obj = sim.solve(verbose = False)

Adding restriction -x1 + x2 - x5 = 1 ...
Done
Adding restriction x1 - x3 + x4 + 2*x5 = 1 ...
Done
Adding objective 2*x1 + 2*x3 + 3*x5 ...
Done
Splitting into positive and negative...
Done
Adding slack/artificial variables...
Done
Setting Phase 1 basis and Phase 2...
Done
Starting simplex...
Simplex finished! 6 iterations with 0.001s to run.
Simplex Status: success
Switching back to original x values...
Done
Final result:
X:
[[0.]
 [1.]
 [0.]
 [1.]
 [0.]]
Objective Value:
[]


In [7]:
# testando o simplex

sim = Simplex('min')                                                      
sim.add_vars(['x1 positive','x2 positive','x3 positive','x4 positive','x5 positive'])   
sim.add_rest(['2*x1 - x2 + 4*x3 - 2*x4 + x5 = 2'])                                    
sim.add_rest(['-x1 - 3*x3 + x4 - x5 = 1'])
sim.add_obj('-3*x1 - x2 + x3 + 4*x4 + 7*x5')
x_final, obj = sim.solve(verbose = False)

Adding restriction 2*x1 - x2 + 4*x3 - 2*x4 + x5 = 2 ...
Done
Adding restriction -x1 - 3*x3 + x4 - x5 = 1 ...
Done
Adding objective -3*x1 - x2 + x3 + 4*x4 + 7*x5 ...
Done
Splitting into positive and negative...
Done
Adding slack/artificial variables...
Done
Setting Phase 1 basis and Phase 2...
Done
Starting simplex...
Simplex finished! 2 iterations with 0.0s to run.
Simplex Status: unfeasible
ERROR: unfeasible
